In [1]:
#Eduardo Darrazão
#Rodrigo Faria

In [58]:
import cv2
import numpy as np
import imutils

In [233]:
#
class Shape_Detector:
        def __init__(self):
            pass
        
        def detect(self, c):
            shape = "desconhecido"
            #Perimetro
            peri = cv2.arcLength(c, True)
            #Ramer–Douglas–Peucker algorithm, simplifica traços
            approx = cv2.approxPolyDP(c, 0.116*peri, True)
             
            #Se tem aproximadamente quatro vértices
            if len(approx) == 4:
                shape = "quadrilatero"
                
            return shape    

In [303]:
def detecta_palavras(img):
    
    #resized = imutils.resize(img, width=300)
    #ratio = img.shape[0] / float(resized.shape[0])
    
    ret, imgthresh = cv2.threshold(img,170,255,cv2.THRESH_BINARY_INV)
    cv2.imwrite('im2_bin.jpg', imgthresh)
        
    #RETR_EXTERNAL: seleciona contornos externos
    #CHAIN_APPROX_TC89_KCOS: Teh-Chin chain, método de aproximação de contorno
    #Determina a região de suporte pra cada ponto,
    #computa curvatura para cada,
    #determina ponto dominante por processo de supressão dos não máximos
    contorno = cv2.findContours(imgthresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
    sd = Shape_Detector()    
    
    h, w = img.shape[:2]
    img_black = np.zeros((h,w), np.uint8)
    
    for c in contorno[0]:
        M = cv2.moments(c)
        shape = sd.detect(c)
        if shape == "quadrilatero" :
            cv2.drawContours(img, [c], -1, (0,255,0), 2)
            cv2.drawContours(img_black, [c], -1, (255,255,255), 2)
            
    cv2.imwrite('im4_contornos.jpg', img_black)
    img_linhas = img_black.copy()
    
    
    
    #Procura blocos
    for altura in range(h):
        for largura in range(w):
            #Se o pixel for branco
            if img_black[altura][largura] == 255:
                contador_fundo = 0
                contador_borda = 0
                new_borda = 0
                new_fundo = 0
                tam_fundo = -1
                tam_espaco = -2
                signal = 0  #1: borda | -1: fundo
                
                for linha in range (largura, w):
                    #se borda
                    if img_black[altura][linha] == 255:
                        contador_borda += 1
                        #se ultimo foi fundo
                        if signal == -1:
                            new_borda += 1
                            
                            #se tamanho do fundo n foi definido
                            if tam_fundo == -1:
                                tam_fundo = contador_fundo
                        signal = 1
                        
                        
                    else: #se fundo
                        contador_fundo += 1
                        #se ultimo foi borda
                        if signal == 1:
                            new_fundo += 1
                            
                            #se primeira iteração de fundo
                            if tam_espaco == -2:
                                tam_espaco += 1;
                            #se tamanho do espaço n foi definido
                            elif tam_espaco == -1:
                                tam_espaco = 
                        signal = -1
                    
                        
                if new_fundo == 16:
                    print "fundo: " + str(contador_fundo) + ", borda: " + str(contador_borda) + "\n"
                    print "new_fundo: " + str(new_fundo) + ", new_borda: " + str(new_borda) + "\n"
                    for abc in range(largura, w):
                        img_linhas[altura][abc] = 255
                        
    cv2.imwrite('im5_linhas.jpg', img_linhas)
    
    #kernel = np.ones((3,3),np.uint8)
    #Dilatação
    #img_dil = cv2.dilate(img_black ,kernel,iterations = 1)    
    #img_dil = cv2.bitwise_not(img_dil)
    #img_black = img_black + img_dil
    #cv2.imwrite('im5_dilat.jpg', img_black)
    
    

    return img

In [309]:
img1 = cv2.imread("1.jpg", cv2.IMREAD_GRAYSCALE )
cv2.imwrite('im1_gray.jpg', img1)

img1 = cv2.GaussianBlur(img1,(3,3),1)

img1c = detecta_palavras(img1)
cv2.imwrite('im3_detect.jpg', img1c)

True